In [2]:
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sys import argv, exit
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [19]:
df = pd.read_csv('../dados/articles.csv')
df

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...
...,...,...,...,...,...,...
167048,"Em cenário de crise, tucano Beto Richa assume ...",O tucano Beto Richa tinha tudo para começar se...,2015-01-01,poder,NaN,http://www1.folha.uol.com.br/poder/2015/01/156...
167049,Filho supera senador Renan Calheiros e assume ...,O economista Renan Filho (PMDB) assume nesta q...,2015-01-01,poder,NaN,http://www1.folha.uol.com.br/poder/2015/01/156...
167050,"Hoje na TV: Tottenham x Chelsea, Campeonato In...",Destaques da programação desta quinta-feira (1...,2015-01-01,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/01/1...
167051,Kim Jong-un diz estar aberto a se reunir com p...,"O líder norte-coreano, Kim Jong-un, disse nest...",2015-01-01,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/01/156...


In [26]:
df.columns

Index(['title', 'text', 'date', 'category', 'subcategory', 'link'], dtype='object')

In [20]:
tokenizer = RegexpTokenizer(r'\w+')
documentos = list()
stpw = {}
for p in stopwords.words("portuguese"):
    stpw[p] = True

for index, row in df.iterrows():
    doc = list()
    for p in tokenizer.tokenize(row['title'].lower()):
        if p not in stpw:
            doc.append(p)
    documentos.append(TaggedDocument(words=doc, tags=['DOC_%s' % index]))

In [23]:
documentos[934].words

['fator', 'lula']

In [66]:
modelo = Doc2Vec(vector_size=100, window=10, min_count=3, alpha=0.025, min_alpha=0.025, train_lbls=False, workers=20)
modelo.build_vocab(documentos)
for epoca in range(10):
    modelo.train(documentos, total_examples=modelo.corpus_count, epochs=1)
    modelo.alpha -= 0.002
    modelo.min_alpha = modelo.alpha

modelo.save('modelos/titulos')

In [5]:
modelo = Doc2Vec.load('modelos/doc2vec.title')

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
#print(modelo.wv.most_similar('DOC', topn=20))
#print(modelo.wv.vocab)
docv = modelo['DOC_76']
sorted(modelo.docvecs.most_similar(positive=[docv], topn=100), key = lambda pos: int(pos[0].split('_')[1]))
cosine_similarity([modelo['DOC_934']], [docv])

array([[0.75255346]], dtype=float32)